In [32]:
import pandas as pd
import numpy as np
import sys
import matplotlib
import matplotlib.pyplot as plt

from sklearn.preprocessing import FunctionTransformer

In [2]:
print("Python version")
print (sys.version)
print("Version info.")
print (sys.version_info)

Python version
3.7.7 (tags/v3.7.7:d7c567b08f, Mar 10 2020, 10:41:24) [MSC v.1900 64 bit (AMD64)]
Version info.
sys.version_info(major=3, minor=7, micro=7, releaselevel='final', serial=0)


# Prepare DataSet

In [3]:
%%script false
from sklearn.datasets import fetch_20newsgroups
mydata  = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'), shuffle=True)
print('Training data size:', len(mydata['data']))

Couldn't find program: 'false'


In [4]:
%%script false
a = [np.unique(['.'.join(i.split('.')[:k]) for i in mydata.target_names]) for k in range(1,4)]
targets = np.unique(mydata.target)
mapping = dict(zip(targets, a[2]))

Couldn't find program: 'false'


In [5]:
import pickle
def save_mapping(mapping_dict, name):
    with open('data/{}.pickle'.format(name), 'wb') as handle:
        pickle.dump(mapping_dict, handle, protocol=pickle.HIGHEST_PROTOCOL)
        
def load_mapping(name):
    with open('data/{}.pickle'.format(name), 'rb') as handle:
        return pickle.load(handle)

In [6]:
%%script false

def generate_new_clusters(obj):
    for v in mapping.values():
        for i in obj:
            if v.startswith(i):
                yield i

mapping2 = dict(zip(targets, generate_new_clusters(a[1])))
mapping3 = dict(zip(targets, generate_new_clusters(a[0])))
print(mapping2)
print(mapping3)
print(len(mapping))
print(len(mapping2))
print(len(mapping3))

Couldn't find program: 'false'


In [7]:
TARGET_COLUMNS = ['clusters', 'clusters_2', 'clusters_3']
TEXT_COLUMN = 'data'

In [8]:
%%script false

df = pd.DataFrame({'data': mydata.data, 'target': mydata.target})
for mapping, col in zip([mapping, mapping2, mapping3], TARGET_COLUMNS):
    df[col] = df['target'].apply(lambda x: mapping[x])
    
df.to_parquet('data/df.parquet')

Couldn't find program: 'false'


# Preproccessing DataSet

In [9]:
%%script false
df = pd.read_parquet('data/df.parquet')

Couldn't find program: 'false'


In [10]:
%%script false
df.shape

Couldn't find program: 'false'


In [11]:
%%script false
def drop_duplicate_values(x):
    return x.drop_duplicates()
df = FunctionTransformer(drop_duplicate_values, validate=False).transform(df)

Couldn't find program: 'false'


In [12]:
%%script false
df.shape

Couldn't find program: 'false'


# Clusters Frequency

In [13]:
def get_frequency(column_name):
    targets, frequency = np.unique(df[column_name], return_counts=True)
    if column_name == 'clusters_3':
        return targets, frequency
    return range(len(targets)), frequency

def plot_bar_clusters(func, title):
    fig, axs = plt.subplots(1, 3, figsize=(19, 3), sharey=False)
    for num, col in enumerate(TARGET_COLUMNS):
        axs[num].bar(*func(col))
    fig.suptitle(title)

In [14]:
%%script false
plot_bar_clusters(get_frequency, 'Clusters Frequency')

Couldn't find program: 'false'


In [15]:
%%script false
from sklearn.preprocessing import LabelEncoder
for col in TARGET_COLUMNS:
    le = LabelEncoder()
    df[col] = le.fit_transform(df[col])    
    save_mapping(dict(enumerate(le.classes_)), col)
df = df.drop(columns=['target'])

Couldn't find program: 'false'


In [16]:
from gensim.parsing.preprocessing import remove_stopwords
from gensim.utils import tokenize

import nltk
from nltk.stem import WordNetLemmatizer
nltk.download('wordnet')
wordnet_lemmatizer = WordNetLemmatizer()

from nltk.sentiment import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
vader_analyzer = SentimentIntensityAnalyzer()

def word_lemmatizer(word):
    word = word.replace('_', '')
    word1 = wordnet_lemmatizer.lemmatize(word, pos = "n") # NOUNS
    word2 = wordnet_lemmatizer.lemmatize(word1, pos = "v") # VERB
    return wordnet_lemmatizer.lemmatize(word2, pos = ("a")) # ADJ
    

def text_lemmatizer(text):
    return ' '.join(map(word_lemmatizer, text))


def lemmatizer(x):
    x[TEXT_COLUMN] = x[TEXT_COLUMN].apply(lambda text: text_lemmatizer(tokenize(remove_stopwords(text))))
    return x

def get_sentimnent(x):
    x['sentimnent'] = x[TEXT_COLUMN].apply(lambda text: vader_analyzer.polarity_scores(text))
    return x

[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Andrei_Gavrilov\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
def text_feature_selector(x):
    x['word_count'] = x[TEXT_COLUMN].apply(lambda text : len(str(text).split()))
    x['length'] = x[TEXT_COLUMN].apply(len)
    x['word_density'] = x['length'] / x['word_count']
    return x

In [18]:
%%script false

df = FunctionTransformer(lemmatizer, validate=False).transform(df)
df = FunctionTransformer(text_feature_selector, validate=False).transform(df)
df = FunctionTransformer(get_sentimnent, validate=False).transform(df)
df = pd.concat([df, pd.json_normalize(df['sentimnent'])], axis=1).drop(columns=['sentimnent'])

df.to_parquet('data/preprocessed_df.parquet')

Couldn't find program: 'false'


In [19]:
df = pd.read_parquet('data/preprocessed_df.parquet')

In [20]:
from functools import partial
def get_word_density(colname, feature):
    tdf = df.groupby([colname]).agg(feature).mean()
    return tdf.index, tdf.values

In [21]:
# plot_bar_clusters(partial(get_word_density, feature='word_density') , 'Clusters word_density')

In [22]:
# plot_bar_clusters(partial(get_word_density, feature='length'), 'Clusters length')

In [23]:
# plot_bar_clusters(partial(get_word_density, feature='word_count'), 'Clusters word_count')

In [24]:
from wordcloud import WordCloud

def clusters_wordcloud(name):
    mapping = load_mapping(name)
    pdf = pd.DataFrame(df.groupby(name).agg('data').sum())

    fig = plt.figure(figsize=(20, 25))
    for i in range(len(mapping)):
        ax = fig.add_subplot(7,3,i+1)
        wordcloud = WordCloud().generate(pdf['data'][i])
        ax.set_title("WordCloud " + mapping[pdf.index[i]])
        ax.imshow(wordcloud, interpolation='bilinear')
        ax.axis('off')

In [25]:
# clusters_wordcloud('clusters')

In [26]:
# clusters_wordcloud('clusters_2')

In [27]:
# clusters_wordcloud('clusters_3')

In [28]:
df.head()

,data,clusters,clusters_2,clusters_3,word_count,length,word_density
0,I sure bashers Pens fan pretty confuse lack ki...,10,8,3,71,400,5.633803
1,My brother market high performance video card ...,3,3,1,39,232,5.948718
2,Finally say dream about Mediterranean That new...,17,14,6,161,926,5.751553
3,Think It s SCSI card DMA transfer NOT disk The...,3,3,1,82,476,5.804878
4,I old Jasmine drive I use new system My unders...,4,3,1,58,342,5.896552
